In [15]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, GRU, LSTM, Masking
import json
import time
import re

In [16]:
with open('./preprocessed_data/questions.json', 'r') as f:
    json_data = json.load(f)
    question_corpus = tokenizer_from_json(json_data)
    f.close()

with open('./preprocessed_data/answers.json', 'r') as f:
    json_data = json.load(f)
    answer_corpus = tokenizer_from_json(json_data)
    f.close()

npzfile = np.load('./preprocessed_data/data.npz') 

In [17]:
q_word2ind={e:i for e,i in question_corpus.word_index.items() if i <= 2500}
q_ind2word={e:i for i,e in q_word2ind.items()}
a_word2ind={e:i for e,i in answer_corpus.word_index.items() if i <= 2500}
a_ind2word={e:i for i,e in a_word2ind.items()}

In [18]:
# define encoder
# notice that the encoder is totally same with the encoder in training model
def Encoder(inputdim, embeddingsize, inputlen, n_units):

    encoder_input = Input((inputlen,))
    encoder_embed = Embedding(inputdim, embeddingsize)(encoder_input)
    encoder_mask = Masking()(encoder_embed)
    encoder = GRU(n_units, return_sequences=True, return_state = True)
    encoder_output,encoder_state = encoder(encoder_mask)
    
    encoder=Model(encoder_input, [encoder_output,encoder_state])
    
    return encoder

In [19]:
class BahdanauAttention(tf.keras.layers.Layer):

    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
    def call(self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [20]:
def Decoder(inputdim, embeddingsize, inputlen, units):

    dec_input = Input((1, ))
    dec_embed = Embedding(inputdim,embeddingsize)(dec_input)
    enc_output = Input((inputlen, units, ))
    hidden = Input((units, ))
    context_vector,attention_weights = BahdanauAttention(units)(hidden, enc_output)

    context_expand = tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, 1))(context_vector)
    full_context = tf.concat([tf.expand_dims(context_vector, 1), dec_embed], axis=-1)

    output, state = GRU(units, return_sequences=True, return_state=True)(full_context)

    flat_output = tf.keras.layers.Flatten()(output)
    decoder_output = Dense(inputdim, activation = 'softmax')(flat_output)

    decoder = Model([enc_output, hidden, dec_input], [decoder_output, state])

    return decoder

In [21]:
VocabSize = 2501
EmbeddingDim = 128
Units = 256
BatchSize = 64
QuestionLen = npzfile['arr_0'].shape[1]
AnswerLen = npzfile['arr_1'].shape[1]

In [22]:
encoder=Encoder(VocabSize,EmbeddingDim,QuestionLen,Units)
decoder=Decoder(VocabSize,EmbeddingDim,QuestionLen,Units)

In [23]:
encoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 8, 128)            320128    
_________________________________________________________________
masking_1 (Masking)          (None, 8, 128)            0         
_________________________________________________________________
gru_2 (GRU)                  [(None, 8, 256), (None, 2 296448    
Total params: 616,576
Trainable params: 616,576
Non-trainable params: 0
_________________________________________________________________


In [24]:
decoder.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 8, 256)]     0                                            
__________________________________________________________________________________________________
bahdanau_attention_1 (BahdanauA ((None, 256), (None, 131841      input_8[0][0]                    
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 1)]          0                                            
____________________________________________________________________________________________

In [25]:
encoder.load_weights('./trained_model/attention_enc_test.h5')
decoder.load_weights('./trained_model/attention_dec_test.h5')

In [26]:
def clean_text(text):

    # remove unnecessary characters in sentences
    
    text = text.lower().strip()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"there's", "there is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r'[" "]+', " ", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [27]:
def evaluate(sentence):
    
    sentence=clean_text(sentence) # clean the input text
    encoder_inputs=[]
    # convert the input text to index sequence and use unk replace the word not in vocabulary
    for word in sentence.split():
        if word in q_word2ind.keys():
            encoder_inputs.append(q_word2ind[word])
        elif word not in q_word2ind.keys():
            encoder_inputs.append(q_word2ind['unk'])
            
    encoder_inputs=tf.keras.preprocessing.sequence.pad_sequences([encoder_inputs],maxlen=QuestionLen,padding='post')
    encoder_inputs = tf.convert_to_tensor(encoder_inputs)
    encoder_output, encoder_state=encoder(encoder_inputs)
    
    hidden_state=encoder_state
    decoder_input=tf.expand_dims([a_word2ind['bos']], 0)
    
    result=''
    for t in range(AnswerLen):
        pred,state=decoder([encoder_output,hidden_state,decoder_input])
        pred=np.squeeze(pred)
        pred_ind=tf.math.argmax(pred).numpy()
        
        if a_ind2word[pred_ind]=='eos':
            return result

        result+=a_ind2word[pred_ind] + ' '
        decoder_input=tf.expand_dims([pred_ind],0)
        hidden_state=state
        
    return result

In [39]:
while True:
    inputs = input('User :> ')
    if inputs == 'quit':
        break

    result = evaluate(inputs)

    print('Bot :> ' + result)

User :>  hello


Bot :> hi 


User :>  where are you going


Bot :> i am not 


User :>  quit
